* Collect a file, capture <filename>
* render the image as html with a form
* submit album, location, caption, tags
* create <filename1>, which is <filename> + N times <filename has been seen>
* capture width, height
* save file to all-photos/<filename1>
* create JSON:
    
    {filename: filename1, width: width, height: height, caption: caption, tags: tags, location:location, album:album>

In [1]:
import json
class JSON(object):
    def __init__(self, path='data/info.json', dirname='data/records'):
        self.path = path
        self.dirname = dirname
        if not os.path.exists(self.path):
            self._make_new()
        
    def _make_new(self):
        with open(self.path, 'w') as f:
            json.dump([], f)
    
    def update(self, value):
        obj = self.get_data()
        obj.append(value)
        with open(self.path, 'w') as f:
            json.dump(obj, f)
            
    def get_data(self):
        with open(self.path, 'r') as f:
            obj = json.load(f)
        return obj
    
    def to_mongo_data(self):
        data = self.get_data()
        for i, doc in enumerate(data):
            ID = doc['image_id']
            with open(os.path.join(self.dirname, "{}.json".format(ID)), 'w') as f:
                json.dump(doc, f)
                
    def get_values(self, field):
        return [i.get(field, None) for i in self.get_data()]

In [2]:
# from ipywidgets import SelectMultiple, Checkbox, Layout, Checkbox
# g = SelectMultiple(options=[1,2,3], layout=Layout(width='200px', height='{}px'.format(20 * 3)))
# #g = Checkbox(description='use last')
# #for i in set(data.get_values('type')):
# #    display(Checkbox(description=i))
# display(g)

In [29]:
import json, os
from ipywidgets import Text, Image, HBox, VBox, Textarea, Layout, Checkbox, SelectMultiple
from IPython.display import display, clear_output
from PIL import Image as pil
from ipywidgets import Button
from ipywidgets import HTML as iHTML

def get_image_data(filename):
    with open(filename, 'rb') as img:
        data = img.read()
    return data

def get_image_dim(filename):
    """returns width, height"""
    with pil.open(filename) as img:
        return img.size
    
def get_image_widget(filename, width=300, height=200):
    """returns width, height"""
    widget = Image(value=get_image_data(filename), width=width, height=height) 
    return widget

def get_video_widget(path):

    video_data = """
    <video width="320" height="240" controls>
      <source src="{}">
    Your browser does not support the video tag.
    </video>
    """.format(path)
    widget = iHTML(value=video_data)
    
    return widget

def flatten(L):
    return [item for sublist in L for item in sublist]

class form(object):
    def __init__(self, paths, data):
        """
        paths needs to be a generator
        """
        self.data = data
        self.paths = paths
        self.target_image_dir = 'public/photos/all-photos'
        self.messages = ""
        self.last = {}
        self.counter = 0
        self.use_last = Checkbox(value=False, description="Use last")        
        self.make_new_form()
        
    @property 
    def seen(self):
        data = self.data.get_data()
        return [i['src'] for i in data]

    def next(self):
        try:
            self.counter += 1
            self.path = next(self.paths)
            if self.path in self.seen:
                self.messages += "skipping {}, already processed".format(self.path)
                self.delete_source_file()
                return self.next()
            else:
                return True
        except StopIteration:
            print("All photos processed! Exiting")
            return False
            
    def get_object_data(self):
        with open(self.path, 'rb') as f:
            data = f.read()
        return data
    
    def scrape_album_entries(self):
        results = list(self.pick_album.value)
        if self.new_album.value:
            for entry in self.new_album.value.split(","):
                results.append(entry.strip())
        return results
        
        
    def get_seen_albums(self):
        return set(filter(lambda x: x is not None, flatten(self.data.get_values('album'))))
        
    def make_new_form(self):
        clear_output()
        self.messagebox = Textarea(self.messages)
        self.messages = ""
        keep_going = self.next()
        if keep_going:
            self.type = self.get_type()
            self.object_data = self.get_object_data()
            print("Currently on {}".format(self.path))
            print("Processed {0}".format(self.counter))
            if self.type in ('video', 'image'):  
                self.use_last = Checkbox(value=self.last.get('use_last', False), description="Use last")
                self.caption = Text(value=self.last.get('caption', ""), description='caption')
                self.location = Text(value=self.last.get('location', ""), description='location')
                self.tags = Text(value=self.last.get('tags', ""), description='tags')
                albums = self.get_seen_albums()
                self.pick_album = SelectMultiple(
                    options=albums, 
                    layout=Layout(width='200px', height='{}px'.format(20 * len(albums)))
                )
                self.pick_album.value = self.last.get('pick_album', tuple())
                self.new_album = Text("", description='Add new album')
                self.skip = Button(description="skip")
                self.skip.on_click(self.save_other)
                self.submit = Button(description="submit")
                self.build_last()
                if self.type == 'image':
                    self.image = get_image_widget(self.path)    
                    self.submit.on_click(self.save_image)                

                elif self.type == 'video':
                    self.image = get_video_widget(self.path)   
                    self.submit.on_click(self.save_video)

                self.button_box = HBox([self.submit, self.skip])                

                self.input_form = VBox([
                    self.use_last,
                    self.caption, 
                    self.location, 
                    self.tags,
                    self.pick_album,
                    self.new_album,
                    self.button_box])

                self.imagebox = VBox([self.image, self.messagebox])
                self.final = HBox([self.input_form, self.imagebox])

                display(self.final)
            else:
                print("Currently on {}".format(self.path))            
                self.messages += "skipping {}, not an image or video\n".format(self.path)
                self.submit = Button(description="submit")
                self.submit.on_click(self.save_other)
                self.messagebox = Textarea(self.messages)


                display(self.messagebox)
                display(self.submit)
                self.build_last()
            
    def build_last(self):
        if self.use_last.value is True:
            self.last = {
                'caption': self.caption.value,
                'location': self.location.value,
                'tags': self.tags.value,
                'use_last': self.use_last.value,
                'pick_album': self.pick_album.value
            }
        else:
            self.last = {
                'use_last': self.use_last.value
            }
    def make_name(self):
        return self.prep_filename(self.parse_filename(self.path))
        
    def parse_filename(self, path):
        return path.split("/")[-1]
        
    def prep_filename(self, filename):
        filename = filename.lower()
        split = filename.split(".")
        ext = split[-1]
        name ="".join(split[:-1])
        #name, ext = filename.split(".")
        n = len(self.seen)
        filename = "img-{0}.{1}".format(n, ext)
        return filename
    
    def save_image_object(self):
        target = os.path.join(self.target_image_dir, self.make_name())
        with open(target, 'wb') as target_file:
            target_file.write(self.get_object_data())
        self.messages += "saved image to {} \n".format(target)
        
    def save_video_object(self):
        target = os.path.join(self.target_image_dir, self.make_name())
        with open(target, 'wb') as target_file:
            target_file.write(self.get_object_data())
        self.messages += "saved video to {} \n".format(target)    
            
    def save_image_form(self):
        self.data.update(self.image_to_form()) 
        self.messages += "updated metadata to {} \n".format(data.path)
        
    def save_video_form(self):
        self.data.update(self.video_to_form()) 
        self.messages += "updated metadata to {} \n".format(data.path)
        
    def save_other_form(self):     
        self.data.update(self.skipped_to_form()) 
        self.messages += "updated metadata with skipped entry to {} \n".format(data.path)  
        
    def delete_source_file(self):
        os.remove(self.path)
        
    def save_image(self, b):
        self.build_last()
        self.save_image_object()
        self.save_image_form()
        self.delete_source_file()
        self.make_new_form()
        
    def save_other(self, b):       
        self.build_last()        
        self.save_other_form()
        self.delete_source_file()
        self.make_new_form()
        
    def save_video(self, b):
        self.build_last()        
        self.save_image_object()
        self.save_video_form()
        self.delete_source_file()
        self.make_new_form()        
        
    def skipped_to_form(self):
        return {
            "display": False,
            "caption": "",
            "tags": "",
            "location": "",
            'image_id': self.make_name(),
            'src': self.path,
            'type': "",
            'width': 0,
            'height': 0,
            'album': "",        
        }        

    def image_to_form(self):
        width, height = get_image_dim(self.path)
        return {
            "display": True,
            "caption": self.caption.value,
            "tags": self.tags.value,
            "location": self.location.value,
            'image_id': self.make_name(),
            'src': self.path,
            'type': 'image',
            'width': width,
            'height': height,
            'album': self.scrape_album_entries()            
        }
    
    def video_to_form(self):
        return {
            "display": True,            
            "caption": self.caption.value,
            "tags": self.tags.value,
            "location": self.location.value,
            'image_id': self.make_name(),
            'src': self.path,
            'type': 'video',
            'width': 0,
            'height': 0,
            'album': self.scrape_album_entries()
        }    
    
    def get_type(self):
        ext = self.path.split(".")[-1]
        if ext.lower() in ('mov', 'm4v', 'mp4'):
            return 'video'
        elif ext.lower() in ('jpg', 'png', 'jpg-3'):
            return 'image'
        else:
            return 'other'

In [30]:
def yield_files(directory):
    for dirpath, dirnames, filenames in os.walk(directory):
        for dirname in dirnames:
            yield_files(dirname)
        for fname in filenames:
            if '.DS_Store' not in fname and 'all-photos' not in dirpath:
                yield os.path.join(dirpath, fname)

In [56]:
print(len(list(yield_files('public/photos/queue'))))

297


In [63]:
import pandas as pd
files = list(yield_files('public/photos/queue'))
records = []
for i in files:
    parts = i.split("/")
    filename = parts[-1]
    file_number = int("".join([j for j in filename if j.isdigit()]))
    records.append({'file':i, 'number':file_number})
files = pd.DataFrame(records).sort_values('number')['file'].values.tolist()
print(len(files))

258


In [46]:
queue = iter(files)
data = JSON()
f = form(queue, data)

Currently on public/photos/queue/IMG_0465.jpg
Processed 86


HBox(children=(VBox(children=(Checkbox(value=True, description='Use last'), Text(value='A gull hiding on the pinnacle of the Pantheon.', description='caption'), Text(value='Rome', description='location'), Text(value='Italy, Rome, Pantheon', description='tags'), SelectMultiple(index=(3,), layout=Layout(height='320px', width='200px'), options=('Holidays in Phoenix', 'Aaron and Kira', 'Marcel', 'Italy', 'Visiting LA', 'George', 'Japan', 'LeeLoo', 'Napa and Sonoma', 'Harry Potter Night', 'Luna', 'String Bean', 'Furry Friends', 'General', 'Taliesin West', 'Kira and Mary Jane'), value=('Italy',)), Text(value='', description='Add new album'), HBox(children=(Button(description='submit', style=ButtonStyle()), Button(description='skip', style=ButtonStyle()))))), VBox(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00H\x00H\x00\x00\xff\xe1\x04\x04Exif\x00\x00MM\x00*\x00\x00\x00\x08\x00\x08\x01\x0f\x00\x02\x00\x00\x00\x06\x00\x00\x00n\x01\x10\x00\x02\x00\x00\x00\t\x00\x00\x00t\x01\x1a\x00\x05\x00\x00\x00\x01\x00\x00\x00~\x01\x1b\x00\x05\x00\x00\x00\x01\x00\x00\x00\x86\x01(\x00\x03\x00\x00\x00\x01\x00\x02\x00\x00\x011\x00\x02\x00\x00\x00\x0b\x00\x00\x00\x8e\x012\x00\x02\x00\x00\x00\x14\x00\x00\x00\x9a\x87i\x00\x04\x00\x00\x00\x01\x00\x00\x00\xae\x00\x00\x00\x00Apple\x00iPhone 6\x00\x00\x00\x00\x00H\x00\x00\x00\x01\x00\x00\x00H\x00\x00\x00\x01Photos 2.0\x00\x002017:10:17 17:44:40\x00\x00!\x82\x9a\x00\x05\x00\x00\x00\x01\x00\x00\x02@\x82\x9d\x00\x05\x00\x00\x00\x01\x00\x00\x02H\x88"\x00\x03\x00\x00\x00\x01\x00\x02\x00\x00\x88\'\x00\x03\x00\x00\x00\x01\x00 \x00\x00\x90\x00\x00\x07\x00\x00\x00\x040221\x90\x03\x00\x02\x00\x00\x00\x14\x00\x00\x02P\x90\x04\x00\x02\x00\x00\x00\x14\x00\x00\x02d\x91\x01\x00\x07\x00\x00\x00\x04\x01\x02\x03\x00\x92\x01\x00\n\x00\x00\x00\x01\x00\x00\x02x\x92\x02\x00\x05\x00\x00\x00\x01\x00\x00\x02\x80\x92\x03\x00\n\x00\x00\x00\x01\x00\x00\x02\x88\x92\x04\x00\n\x00\x00\x00\x01\x00\x00\x02\x90\x92\x07\x00\x03\x00\x00\x00\x01\x00\x03\x00\x00\x92\t\x00\x03\x00\x00\x00\x01\x00\x10\x00\x00\x92\n\x00\x05\x00\x00\x00\x01\x00\x00\x02\x98\x92\x14\x00\x03\x00\x00\x00\x04\x00\x00\x02\xa0\x92|\x00\x07\x00\x00\x01\x0c\x00\x00\x02\xa8\x92\x91\x00\x02\x00\x00\x00\x04942\x00\x92\x92\x00\x02\x00\x00\x00\x04942\x00\xa0\x00\x00\x07\x00\x00\x00\x040100\xa0\x01\x00\x03\x00\x00\x00\x01\x00\x01\x00\x00\xa0\x02\x00\x04\x00\x00\x00\x01\x00\x00\t\xef\xa0\x03\x00\x04\x00\x00\x00\x01\x00\x00\x06\x00\xa2\x17\x00\x03\x00\x00\x00\x01\x00\x02\x00\x00\xa3\x01\x00\x07\x00\x00\x00\x01\x01\x00\x00\x00\xa4\x01\x00\x03\x00\x00\x00\x01\x00\x02\x00\x00\xa4\x02\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\xa4\x03\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\xa4\x05\x00\x03\x00\x00\x00\x01\x00\x1d\x00\x00\xa4\x06\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\xa42\x00\x05\x00\x00\x00\x04\x00\x00\x03\xb4\xa43\x00\x02\x00\x00\x00\x06\x00\x00\x03\xd4\xa44\x00\x02\x00\x00\x00"\x00\x00\x03\xda\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\xe9\x00\x00\x00\x0b\x00\x00\x00\x052017:10:17 17:44:40\x002017:10:17 17:44:40\x00\x00\x00X\xa8\x00\x00\x0bG\x00\x00\x1ey\x00\x00\re\x00\x00\x01\xb7\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00S\x00\x00\x00\x14\x03\xd7\x06\x94\x02b\x02dApple iOS\x00\x00\x01MM\x00\n\x00\x01\x00\t\x00\x00\x00\x01\x00\x00\x00\x05\x00\x03\x00\x07\x00\x00\x00h\x00\x00\x00\x8c\x00\x04\x00\t\x00\x00\x00\x01\x00\x00\x00\x01\x00\x05\x00\t\x00\x00\x00\x01\x00\x00\x00\x80\x00\x06\x00\t\x00\x00\x00\x01\x00\x00\x00}\x00\x07\x00\t\x00\x00\x00\x01\x00\x00\x00\x01\x00\x08\x00\n\x00\x00\x00\x03\x00\x00\x00\xf4\x00\n\x00\t\x00\x00\x00\x01\x00\x00\x00\x02\x00\x0e\x00\t\x00\x00\x00\x01\x00\x00\x00\x04\x00\x14\x00\t\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00bplist00\xd4\x01\x02\x03\x04\x05\x06\x07\x08UflagsUvalueUepochYtimescale\x10\x01\x13\x00\x00]\xb8\xfa+ce\x10\x00\x12;\x9a\xca\x00\x08\x11\x17\x1d#-/8:\x00\x00\x00\x00\x00\x00\x01\x01\x00\x00\x00\x00\x00\x00\x00\t\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00?\xff\xff\xf3h\x00\x00\r\x

In [59]:
data.to_mongo_data()

In [60]:
import os
try:
    from pymongo import MongoClient
except ModuleNotFoundError:
    !conda install -y pymongo
    from pymongo import MongoClient
with open('.mongo/.credentials') as f:
    lines = f.readlines()
    db_url = list(filter(lambda x: 'db_url' in x,lines))[0].split("=")[1].strip()
client = MongoClient(db_url)
db = client['kira-photos']
collection = db.get_collection('photos')
records = [i for i in collection.find({})]

json_filenames = list(os.walk(data.dirname))[0][2]
list_of_stored_image_ids = [i['image_id'] for i in records]
files_to_write = []
for filename in json_filenames:
    filepath = os.path.join(data.dirname, filename)
    with open(filepath) as f:
        json_file = json.load(f)
    
    if json_file['image_id'] not in list_of_stored_image_ids:
        files_to_write.append(json_file)
collection.insert_many(files_to_write)

In [361]:
obj = data.get_data()

In [363]:
with open(data.path, 'w') as f:
    json.dump(obj, f)

In [349]:
!cp data/info.json data/info-backup.json 

In [ ]:

ls -1 data/*.json | sed 's/.json$//' | while read col; do 
    mongoimport -d db_name -c $col < $col.json; 
done

In [ ]:
cd data/records
ls -1 *.json | while read fname; do  
    mongoimport -h ds113826.mlab.com:13826 -d kira-photos -c photos -u aikramer -p m24WAGli8OXs --file $fname
done